In [ ]:
import os
from urllib.parse import urljoin
import pystac
from pystac.utils import str_to_datetime
from pystac.extensions.eo import EOExtension
from pystac.extensions.sar import SarExtension
from pystac.extensions.sat import SatExtension
from pystac.collection import Summaries

import requests
from requests.auth import HTTPBasicAuth


import datetime
from pystac.extensions.item_assets import AssetDefinition, ItemAssetsExtension
from pystac.extensions.projection import ProjectionExtension
from pystac.extensions.raster import RasterExtension
from pystac.extensions.sat import SatExtension
from pystac.extensions.scientific import Publication, ScientificExtension
from pystac.extensions.timestamps import TimestampsExtension
from pystac.utils import datetime_to_str, str_to_datetime

from pystac.extensions.datacube import (
    DatacubeExtension,
    DimensionType,
    HorizontalSpatialDimensionAxis,
)
from shapely.geometry import box, mapping



import os
from dotenv import load_dotenv

# .env-Datei laden
load_dotenv('za.env')

# Variablen abrufen
username = os.getenv("username_prod")
password = os.getenv("password_prod")

print("Username:", username)


def create_collection():

    """Creation of STAC collection"""

    datacube = {
    "x": {
        "axis": HorizontalSpatialDimensionAxis.X,
        "type": DimensionType.SPATIAL,
        "extent": [112500, 695500],
    },
    "y": {
        "axis": HorizontalSpatialDimensionAxis.Y,
        "type": DimensionType.SPATIAL,
        "extent": [258500, 586500],
    },
    "time": {
    "type": DimensionType.TEMPORAL,
    "extent": ["1961-01-01T00:00:00Z", None],
},
}

    parameters = {
        "TN": {
            "description": "minimum temperature",
            "unit": "degree Celsius",
        },
        "TX": {
            "description": "maximum temperature",
            "unit": "degree Celsius",
        },
        "RR": {
            "description": "relative humidity",
            "unit": "p",
        },
        "RR": {
            "description": "precipitation sum",
            "unit": "kg m-2",
        },
        "SA": {
            "description": "duration of sunshine",
            "unit": "s",
        },
    }

    collection = pystac.Collection(
        id="SPARTACUS",
        title= "Spartacus Analysis Daily (1 km)",
        description="This gridded dataset, stored in Zarr format, contains daily observations of air temperature (minimum: TN, maximum: TX), precipitation (RR), and sunshine duration (SA) over Austria at 1 km resolution, starting in 1961. It also includes monthly, seasonal, and annual aggregates. The dataset is operationally updated on a daily basis.",
        extent=pystac.Extent(
            pystac.SpatialExtent([[9.393909, 46.161327, 17.38218, 49.17754]]),
            pystac.TemporalExtent([[str_to_datetime("1961-01-01T00:00:00Z"), None]]),
        ),
        keywords=["GeoSphere", "Spartacus", "sunshine", "pressure", "precipitation", "temperature"],
        license="CC-BY-4.0",
        extra_fields={"cube:dimensions": datacube,
                      "cube:variables": parameters},
    )

    collection.providers = [
    pystac.Provider(
        name="EODC",
        roles=[
            pystac.ProviderRole.HOST,
        ],
        url="https://eodc.eu/",
        ),
        pystac.Provider(
        name="GeoSphere Austria",
        roles=[
            pystac.ProviderRole.PRODUCER,
            pystac.ProviderRole.LICENSOR,
            pystac.ProviderRole.PROCESSOR,
        ],
        url="https://data.hub.geosphere.at/",
    ),
]

    collection.stac_extensions = [
        SarExtension.get_schema_uri(),
        SatExtension.get_schema_uri(),
        EOExtension.get_schema_uri(),
    ]

    summaries_dict = {}
    summaries_dict["Doi"] = ["https://doi.org/10.60669/m6w8-s545"]
    summaries_dict["Timezone"] = ["UTC"]
    summaries_dict["Projection"] = ["ETRS89 (EPSG: 4258)"]
    summaries_dict["Spatial Resolution"] = ["1 km"]
    summaries_dict["Temporal Resolution"] = ["daily"]
    collection.summaries = Summaries(summaries_dict)

    bbox = [9.393909, 46.161327, 17.38218, 49.17754]
    item = pystac.Item(id="dataset", geometry=mapping(box(*bbox)), bbox=bbox, datetime=datetime.datetime.utcnow(), 
                       properties={ "proj:wkt2": '''PROJCS["ETRS89 / Austria Lambert",GEOGCS["ETRS89",DATUM["European_Terrestrial_Reference_System_1989",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],
                                   AUTHORITY["EPSG","6258"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.01745329251994328,AUTHORITY["EPSG","9122"]],
                                   AUTHORITY["EPSG","4258"]],UNIT["metre",1,AUTHORITY["EPSG","9001"]],PROJECTION["Lambert_Conformal_Conic_2SP"],
                                   PARAMETER["standard_parallel_1",49],PARAMETER["standard_parallel_2",46],PARAMETER["latitude_of_origin",47.5],
                                   PARAMETER["central_meridian",13.33333333333333],PARAMETER["false_easting",400000],
                                   PARAMETER["false_northing",400000],AUTHORITY["EPSG","3416"],AXIS["Y",EAST],AXIS["X",NORTH"]]''',})
    item.add_asset("data",
                   pystac.Asset(href="https://data.eodc.eu/collections/SPARTACUS/SPARTACUS.zarr",
                                media_type=pystac.MediaType.ZARR,
                                roles=["data"],
                                title="SPARTACUS",),)
 

    destination = "simon_demo/staccoll"
    collection_path = os.path.join(destination, "{}.json".format(collection.id))
    collection.set_self_href(collection_path)
    collection.save_object()

    collection.add_asset(
    key="thumbnail",
    asset=pystac.Asset(
        href="https://raw.githubusercontent.com/katharinastarzer21/repository_bsp/main/january_2024_map.png",
        media_type=pystac.MediaType.PNG,
        roles=["thumbnail"],
        title="Thumbnail"
    )
)
    

    return collection, item

## Create Collection
create_collection()
k, item = create_collection()
k.to_dict()

app_host = "https://dev.stac.eodc.eu/ingestion/v1"
url = urljoin(app_host, "/collections")



r = requests.post(app_host + "/collections", json=k.to_dict(), auth=HTTPBasicAuth(username, password), timeout = 30)
r = requests.post("https://dev.stac.eodc.eu/ingestion/v1/collections/SPARTACUS/items", json=item.to_dict(), auth=HTTPBasicAuth(username, password), timeout = 30)

#r = requests.delete('https://dev.stac.eodc.eu/ingestion/v1/collections/SPARTACUS',auth=HTTPBasicAuth(username, password), timeout = 30)

print(r.text)


Username: ingest1
{"bbox":[9.393909,46.161327,17.38218,49.17754],"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[17.38218,46.161327],[17.38218,49.17754],[9.393909,49.17754],[9.393909,46.161327],[17.38218,46.161327]]]},"properties":{"datetime":"2025-07-22T08:45:17.953585Z"},"id":"dataset","stac_version":"1.1.0","assets":{"data":{"href":"https://data.eodc.eu/collections/SPARTACUS/SPARTACUS.zarr","type":"application/vnd+zarr","title":"SPARTACUS","roles":["data"]}},"links":[{"rel":"collection","type":"application/json","href":"https://dev.stac.eodc.eu/ingestion/v1/collections/SPARTACUS"},{"rel":"parent","type":"application/json","href":"https://dev.stac.eodc.eu/ingestion/v1/collections/SPARTACUS"},{"rel":"root","type":"application/json","href":"https://dev.stac.eodc.eu/ingestion/v1/"},{"rel":"self","type":"application/geo+json","href":"https://dev.stac.eodc.eu/ingestion/v1/collections/SPARTACUS/items/dataset"}],"stac_extensions":[],"collection":"SPARTACUS"}


In [12]:
print(username)

ingest1


In [ ]:
# Zarr Item erstellen


# Item zur Collection hinzufügen
